In [1]:
%load_ext autoreload
%autoreload 1

%aimport basics
%aimport nn_config
%aimport enc_dec


from basics import *
from nn_config import *
from enc_dec import *
%matplotlib inline

callhome es-en word level configuration
translating es to en
vocab size, en=51, fr=51


In [2]:
xp = cuda.cupy if gpuid >= 0 else np

In [3]:
text_data = pickle.load(open(text_data_dict, "rb"))

In [4]:
model = SpeechEncoderDecoder(SPEECH_DIM, vocab_size_en, num_layers_enc, num_layers_dec,
                               hidden_units, gpuid, attn=use_attn)
if gpuid >= 0:
    cuda.get_device(gpuid).use()
    model.to_gpu()

In [6]:
log_train_fil_name, text_fname, dev_fname, test_fname

('es_speech_to_en_char_model/train_17394sen_4-2layers_128units_es_en_speech2text_batches_callhome_es_en_1.log',
 {'en': '../../corpora/callhome/uttr_fa_vad_wavs/train.en',
  'fr': '../../corpora/callhome/uttr_fa_vad_wavs/speech_train.es'},
 {'en': '../../corpora/callhome/uttr_fa_vad_wavs/dev.en',
  'fr': '../../corpora/callhome/uttr_fa_vad_wavs/speech_dev.es'},
 {'en': '../../corpora/callhome/uttr_fa_vad_wavs/test.en',
  'fr': '../../corpora/callhome/uttr_fa_vad_wavs/speech_test.es'})

In [20]:
%aimport nmt_trials
from nmt_trials import *

Starting experiment
num sentences=10 and num epochs=10


In [21]:
buckets, bucket_lengths = populate_buckets()

  1%|          | 87/13137 [00:00<00:15, 865.98it/s]

Splitting data into 32 buckets, each of width=25


100%|██████████| 13137/13137 [00:06<00:00, 2060.06it/s]

Saving bucket data


In [25]:
batch_training(BATCH_SIZE, BATCH_SIZE, buckets, bucket_lengths, SPEECH_BUCKET_WIDTH, 0)

  0%|          | 0/20 [00:00<?, ?it/s]

Starting experiment
num sentences=10 and num epochs=10


ValueError: All arrays must have same shape except the axis to concatenate

In [13]:
# predict(s=0, num=1, cat="train", display=True, plot=False, p_filt=0, r_filt=0)

In [14]:
# print(b" ".join(get_ids(text_data["train"]["041.004"]["en"])[0]))

In [16]:
# forward_states = model[model.lstm_enc[-1]].h
# backward_states = model[model.lstm_rev_enc[-1]].h

In [17]:
# model.enc_states = F.concat((forward_states, backward_states), axis=1)

In [18]:
# train_loop(num_training=1000, num_epochs=1)

In [19]:
wavs = []
wavs.append(xp.load(os.path.join(speech_dir, "041.001"+speech_extn)))
wavs.append(xp.load(os.path.join(speech_dir, "041.004"+speech_extn)))

In [20]:
[l.shape for l in wavs]

[(1168, 120), (484, 120)]

In [21]:
zero_arr = xp.zeros((3,120), dtype=xp.float32)

In [22]:
wavs[1].dtype, zero_arr.dtype

(dtype('float32'), dtype('float32'))

In [23]:
# xp.vstack((wavs[1], zero_arr))
wavs[1].shape, zero_arr.shape

((484, 120), (3, 120))

In [25]:
wavs.append(pad_array(wavs[1], 1000, at_start=True))

484 120
516
(516, 120)
(484, 120) (516, 120)


In [26]:
xp.concatenate((wavs[1], zero_arr), axis=0).shape

(487, 120)

In [27]:
wavs[1][:3].shape

(3, 120)

In [28]:
xp.pad(wavs[1], pad_width=(0,100), mode='constant', constant_values=0)

array([[-5.05732393, -4.69633675, -4.35915041, ...,  0.        ,
         0.        ,  0.        ],
       [-5.41583586, -4.43796682, -4.59824276, ...,  0.        ,
         0.        ,  0.        ],
       [-4.83030939, -5.28592539, -4.88475227, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [35]:
def display_buckets(bucket_lengths, width_b = SPEECH_BUCKET_WIDTH):
    headings = ("ix", "len b", "num", "max fr", "avg fr", "max en", "avg en")
    print("{0:3s} | {1:5s} | {2:5s} | {3:6s} | {4:8s} | {5:6s} | {6:8s}".format(*headings))
    print("\n".join(["{0:3d} | {1:5d} | {2:5d} | {3:6d} | {4:8.0f} | {5:6d} | {6:8.0f}".format(i[0], (i[0]+1)*width_b, *i[1]) for i in list(bucket_lengths.items())]))


def populate_buckets(width_b = SPEECH_BUCKET_WIDTH, 
                     num_b = SPEECH_NUM_BUCKETS, 
                     speech=True, 
                     num_sent=NUM_TRAINING_SENTENCES,
                     filname_b=speech_bucket_data_fname,
                     cat="train", display=False):
    
    buckets = [[] for i in range(num_b)]
    
    print("Splitting data into {0:d} buckets, each of width={1:d}".format(num_b, width_b))

    with tqdm(total=num_sent) as pbar:
        for i, sp_fil in enumerate(sorted(list(text_data[cat].keys()))[:num_sent]):

            fr_ids, en_ids, speech_feat = get_data_item(sp_fil, cat=cat)

            len_en = len(en_ids)
            len_fr = len(fr_ids)
            len_speech = len(speech_feat)
            
            indx_b = min(num_b-1, len_speech // width_b)

            buckets[indx_b].append((sp_fil, len_speech, len_fr, len_en))

            pbar.update(1)
    
    bucket_lengths = {i:(len(l), 
                    max(l, key=lambda t:t[2])[2],
                    np.mean([i[2]for i in l]),
                    max(l, key=lambda t:t[3])[3],
                    np.mean([i[3]for i in l]))
                     for i, l in enumerate(buckets)}

    if display:
        display_buckets(bucket_lengths)

    # Saving bucket data
    print("Saving bucket data")
    pickle.dump(buckets, open(filname_b, "wb"))
    return buckets, bucket_lengths

In [36]:
b, bucket_lengths = populate_buckets(display=True)

  1%|          | 109/13137 [00:00<00:11, 1086.94it/s]

Splitting data into 32 buckets, each of width=25


100%|██████████| 13137/13137 [00:06<00:00, 2095.63it/s]

ix  | len b | num   | max fr | avg fr   | max en | avg en  
  0 |    25 |   274 |     10 |        3 |     10 |        3
  1 |    50 |  1582 |     21 |        4 |     24 |        4
  2 |    75 |  1167 |     28 |        8 |     31 |        8
  3 |   100 |  1055 |     36 |       14 |     49 |       14
  4 |   125 |   893 |     49 |       19 |     43 |       19
  5 |   150 |   751 |     47 |       24 |     49 |       23
  6 |   175 |   684 |     54 |       29 |     59 |       29
  7 |   200 |   676 |     62 |       34 |     65 |       33
  8 |   225 |   580 |     72 |       38 |     73 |       38
  9 |   250 |   501 |     81 |       43 |     86 |       41
 10 |   275 |   478 |     77 |       48 |     82 |       46
 11 |   300 |   408 |     83 |       53 |     89 |       52
 12 |   325 |   383 |     95 |       55 |     88 |       53
 13 |   350 |   334 |    104 |       61 |    106 |       58
 14 |   375 |   324 |    127 |       67 |    113 |       65
 15 |   400 |   289 |    111 |       71 

In [127]:
b[1][:5]

[('041.002', 36, 3, 3),
 ('041.005', 49, 3, 3),
 ('041.017', 29, 3, 3),
 ('041.020', 36, 9, 9),
 ('041.034', 49, 5, 5)]

In [38]:
haha = [0 for i in range(10)]

In [40]:
d = haha[:30]
print(len(d))

10


In [37]:
bucket_lengths

{0: (274, 10, 2.5547445255474455, 10, 3.2043795620437958),
 1: (1582, 21, 3.9007585335018962, 24, 4.2465233881163087),
 2: (1167, 28, 7.9905741216795203, 31, 8.1045415595544128),
 3: (1055, 36, 13.798104265402843, 49, 13.871090047393364),
 4: (893, 49, 18.660694288913774, 43, 18.548712206047032),
 5: (751, 47, 23.62450066577896, 49, 23.115845539280958),
 6: (684, 54, 29.07456140350877, 59, 28.846491228070175),
 7: (676, 62, 33.80473372781065, 65, 32.655325443786985),
 8: (580, 72, 38.365517241379308, 73, 37.881034482758622),
 9: (501, 81, 42.628742514970057, 86, 41.085828343313374),
 10: (478, 77, 47.55857740585774, 82, 45.937238493723846),
 11: (408, 83, 53.105392156862742, 89, 51.615196078431374),
 12: (383, 95, 55.0, 88, 53.065274151436029),
 13: (334, 104, 61.353293413173652, 106, 58.455089820359284),
 14: (324, 127, 66.589506172839506, 113, 64.503086419753089),
 15: (289, 111, 70.612456747404849, 119, 68.543252595155707),
 16: (258, 117, 74.68217054263566, 132, 72.523255813953483)